# Overview

![Tensorflow Lite](https://www.tensorflow.org/lite/images/convert/workflow.svg)

# API Level

**Basic Requirement: tensorflow version >= 1.9.0**

* `tf.contrib.lite.TFLiteConverter`: convert tensorflow models to tensor flow lite
    * `TFLiteConverter` provides class methods based on the original format of the model.
    * `TFLiteConverter.from_session()` is available for GraphDefs.
    * `TFLiteConverter.from_saved_model()` is available for SavedModels.
    * `TFLiteConverter.from_keras_model_file()` is available for `tf.Keras` files.
* `tf.contrib.lite.Interpreter`: calling Python interpreter

# Converting Example

## Exporting a GraphDef from tf.Session

Convert a Tensorflow GraphDef into a Tensorflow Lite Flatbuffer from a `tf.Session` object.

In [1]:
# if you encounter module 'tensorflow.contrib.lite.python.lite' has no attribute 'TFLiteConverter'
!pip install tf_nightly

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import tensorflow as tf

def sessionExport():
    img = tf.placeholder(name="img", dtype=tf.float32, shape=(1, 64, 64, 3))
    var = tf.get_variable("weights", dtype=tf.float32, shape=(1, 64, 64, 3))
    val = img + var
    out = tf.identity(val, name="out")

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        converter = tf.contrib.lite.TFLiteConverter.from_session(sess, [img], [out])
        tflite_model = converter.convert()
        open("converted_model.tflite", "wb").write(tflite_model)
        
#sessionExport()

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Exporting a GraphDef from file

Convert a Tensorflow GraphDef stored in a file (`.pb` or `.pbtxt`) into Tensorflow Lite FlatBuffer.

You can download a frozen example model from 
https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_1.0_224_frozen.tgz

if you prepare the your own model, please make sure using [freeze_graph.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/freeze_graph.py) to freeze the model.

In [3]:
import tensorflow as tf

def frozenExport():
    graph_def_file = "./mobilenet_v1_1.0_224/frozen_graph.pb"
    input_arrays = ["input"]
    output_arrays = ["MobilenetV1/Predictions/Softmax"]

    converter = tf.contrib.lite.TFLiteConverter.from_frozen_graph(graph_def_file, input_arrays, output_arrays)
    tflite_model = converter.convert()
    open("converted_model.tflite", "wb").write(tflite_model)
    
#frozenExport()

## Exporting a SavedModel

`SaveModel` is the checkpoint-based way to conserve the model. Usually it is the folder conversing a checkpoint, and lots of `.ckpt` or `ckpt-{epoch}` style filename of files.

For more complex SavedModels, the optional parameters that can be passed into `TFLiteConverter.from_saved_model()` are `input_arrays`, `input_shapes`, `output_arrays`, `tag_set` and `signature_key`. Details of each parameter are available by running `help(tf.contrib.lite.TFLiteConverter)`.

In [5]:
import tensorflow as tf

def ckptExport():
    converter = tf.contrib.lite.TFLiteConverter.from_saved_model(saved_model_dir)
    tflite_model = converter.convert()
    open("converted_model.tflite", "wb").write(tflite_model)

#ckptExport()

## Export a tf.keras file

In [7]:
# make sure you have installed h5py
!pip install h5py

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import tensorflow as tf

def h5Export():
    converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file("keras_model.h5")
    tflite_model = converter.convert()
    open("converted_model.tflite", "wb").write(tflite_model)
    
#h5Export()

## Export a tf.keras network

You have to export the `.h5` file first and then convert it into tensorflow lite.

In [9]:
import numpy as np
import tensorflow as tf

def kerasExport():
    # Generate tf.keras model.
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(2, input_shape=(3,)))
    model.add(tf.keras.layers.RepeatVector(3))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(3)))
    model.compile(loss=tf.keras.losses.MSE,
                  optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
                  metrics=[tf.keras.metrics.categorical_accuracy],
                  sample_weight_mode='temporal')

    x = np.random.random((1, 3))
    y = np.random.random((1, 3, 3))
    model.train_on_batch(x, y)
    model.predict(x)

    # Save tf.keras model in HDF5 format.
    keras_file = "keras_model.h5"
    tf.keras.models.save_model(model, keras_file)

    # Convert to TensorFlow Lite model.
    converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file(keras_file)
    tflite_model = converter.convert()
    open("converted_model.tflite", "wb").write(tflite_model)

#kerasExport()

# Adcanced Converting

## Quantized GraphDef

In [10]:
import tensorflow as tf

def quantExport():
    img = tf.placeholder(name="img", dtype=tf.float32, shape=(1, 64, 64, 3))
    const = tf.constant([1., 2., 3.]) + tf.constant([1., 4., 4.])
    val = img + const
    out = tf.fake_quant_with_min_max_args(val, min=0., max=1., name="output")

    with tf.Session() as sess:
        converter = tf.contrib.lite.TFLiteConverter.from_session(sess, [img], [out])
        # you have to transform int into QUANTIZED_UINT8 before exporting
        converter.inference_type = tf.contrib.lite.constants.QUANTIZED_UINT8
        input_arrays = converter.get_input_arrays()
        converter.quantized_input_stats = {input_arrays[0] : (0., 1.)}  # mean, std_dev
        tflite_model = converter.convert()
        open("converted_model.tflite", "wb").write(tflite_model)
        
#quantExport()

# Tensorflow Lite Python Interpreter

## Load and use a `.tflite` model

In [28]:
import numpy as np
import os
import tensorflow as tf

def tfLitePyInterpreter(tflitefile):
    # Load TFLite model and allocate tensors.
    interpreter = tf.contrib.lite.Interpreter(model_path=tflitefile)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Test model on random input data.
    input_shape = input_details[0]['shape']
    input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    print(input_shape)
    print(output_details[0]['shape'])

    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print(np.argmax(output_data, 1))
    
tflitefile = os.path.join("/Users/jiankaiwang/Google_Drive_Devops_Sync/sophia/tmp/converted_model.tflite")
assert os.path.exists(tflitefile), "no such tflite file"
tfLitePyInterpreter(tflitefile)

[  1 224 224   3]
[   1 1001]
[311]
